In [ ]:
!pip install ktrain

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"malayajrath","key":"3dc86e3b6d564b061d883ed51b2632e9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 66% 17.0M/25.7M [00:00<00:00, 50.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 65.1MB/s]


In [ ]:
from zipfile import ZipFile
name='imdb-dataset-of-50k-movie-reviews.zip'
with ZipFile(name,'r') as zip:
  zip.extractall()

In [4]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup

In [ ]:
df=pd.read_csv('/content/IMDB Dataset.csv')


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
sb=SnowballStemmer( language='english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
def remove_stopword_and_stemming(text):
  stopWord=stopwords.words('english')
  stopWord.remove('not')
  text = text.split()
  r=[sb.stem(word) for word in text if word not in set(stopWord)]
  r=" ".join(r)
  return r

In [ ]:
df['review']=df['review'].apply(lambda x: str(x))
df['review']=df['review'].apply(lambda x: x.lower())
df['review']=df['review'].apply(lambda x : re.sub("'",' ',x))
df['review']=df['review'].apply(lambda x :strip_html(x) )
df['review']=df['review'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]'," ",x))  
df['review']=df['review'].apply(lambda x :remove_between_square_brackets(x) )
df['review']=df['review'].apply(lambda x :remove_stopword_and_stemming(x) )

In [ ]:
df.head()

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,positive


In [ ]:
df.to_csv('final_df.csv',index=False)

In [5]:
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
final_df=pd.read_csv('/content/final_df.csv')

In [ ]:
train_df=final_df.iloc[:40000,:]
test_df=final_df.iloc[40000:,:]

In [ ]:
test_df.head()

,review,sentiment
40000,first want say lean liber polit scale found mo...,negative
40001,excit see sitcom would hope repres indian cand...,negative
40002,look cover read stuff entir differ type movi c...,negative
40003,like mani other count appear denni hopper make...,negative
40004,movi v day enjoy first georg jungl good comedi...,negative


(10000,)

In [ ]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_df( train_df=train_df,text_column='review',label_columns='sentiment',val_df=test_df,max_features=20000
                   ,maxlen=400, preprocess_mode='bert')


downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model=text.text_classifier(name='bert',train_data=(x_train,y_train),preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
learner=ktrain.get_learner(model=model,train_data=(x_train,y_train),val_data=(x_test,y_test),batch_size=6)

In [ ]:
# learner.lr_find()
#learner.lr_plot()
lr_rate=2e-5

In [ ]:
learner.fit_onecycle(lr=lr_rate,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
6667/6667 [==============================] - 5892s 884ms/step - loss: 0.3348 - accuracy: 0.8498 - val_loss: 0.2277 - val_accuracy: 0.9063


In [ ]:
predictor=ktrain.get_predictor(learner.model,preproc=preprocess)


In [8]:
data=["this movie was good"]

In [ ]:
predictor.predict(data)

['positive']

In [ ]:
predictor.save('bert_sentimentAnalaysis_model')

In [6]:
reloaded_predictor = ktrain.load_predictor('model')


In [9]:
reloaded_predictor.predict(data)

['positive']